<h1>Titanic - Voorspellen van overlevenden</h1>

In [ ]:
#All imports:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

<h3>De data</h3>
Er zijn 2 data sets klaar gezet, 1 om het model te trainen en 1 om mee te testen. De set om mee te trainen heeft al de overlevings voorspelling per passagier terwijl de test set deze niet heeft 

In [ ]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
combine = [train_df, test_df]

In [ ]:
print(train_df.columns.values)

<h3>Beschrijving van de features</h3>
<ul>
    <li>Categorical: Survived, Sex en Embarked</li>
    <li>Ordinal: Pclass</li>
    <li>Numerical: Age, Fare, SibSp en Parch</li>
    <li>Alpha: Ticket en Cabin</li>
</ul>

In [ ]:
# preview the data
print(train_df.shape)
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

<h4>Typefouten</h4>
<ul>
    <li>Name</li>
</ul>
Is hier een aanpassing voor nodig?

<h4>Null en lege waarden</h4>
<ul>
    <li>Cabin</li>
    <li>Age</li>
</ul>
Is hier een aanpassing voor nodig?


In [ ]:
#Numerical values:
train_df.describe()

<h3>Beschrijving van de data</h3>
<h4>Distributie van de numerieke waarden</h4>
<ul>
    <li>891 passagiers in de training set</li>
    <li>Waardes in "Survived" is 0 | 1</li>
    <li>Ongeveer 38% van deze set heeft het overleefd</li>
    <li>> 75% reisde zonder ouders/kinderen</li>
</ul>

In [ ]:
#Categorical data:
# TODO: describe de categorische data

<h4>Beschrijving van de categorische data</h4>
<ul>
    <li>891 unieke namen</li>
    <li>2 unieke waardes voor Sex, 65% man</li>
    <li>147 unieke cabins</li>
    <li>De meesten zijn ingestapt op locatie S (Southampton)</li>
    <li>Tickets bevatten ongeveer 24% aan duplicaten:</li>
</ul>

In [ ]:
train_df['Ticket'].value_counts()[lambda x: x>1]

<h4>Te valideren aannames</h4>
<ul>
    <li><b>Correlatie</b> Wat is de correlatie van elke feature tov Survival.</li>
    <li><b>Data aanvulling</b>
        <ul>
            <li>Moeten we/kunnen we Age aanvullen?</li>
            <li>Wat doen we met Embarked?</li>
        </ul>
    </li>
    <li><b>Data correctie</b>
        <ul>
            <li>Kunnen we Ticket als feature laten afvallen?</li>
            <li>Kunnen we de Cabin feature uberhaupt gebruiken?</li>
            <li>Passenger Id heeft geen toegevoegde waarde en kan worden laten vallen.</li>
            <li>Wat te doen met de Name feature? Kan een naam een correlatie hebben met de survival rate?</li>
        </ul>
    </li>
    <li><b>Feature creation</b> Kunnen we een aantal features expliciet toevoegen?
        <ul>
            <li>Bijvoorbeeld een feature Familie gebaseerd op de features Parch en SibSp?</li>
            <li>Kunnen we de titel van een persoon uit het naam veld halen en deze gebruiken als nieuwe feature?</li>
            <li>Heeft het zin om op leeftijd te groeperen? Dus om een ordinale feature te maken van de numerieke leeftijds (Age) feature?</li>
        </ul>
    </li>
    <li><b>Generieke aannames:</b> 
        <ul>
            <li>Vrouwen hebben een grotere kans op overleving</li>
            <li>Kinderen jonger dan ... hebben een grotere kans op overleving</li>
            <li>Upper-class passagiers hebben een grotere kans op overleving</li>
        </ul>
    </li>
</ul>

<h4>Correlatie</h4>
Eerst gaan we de correlatie van de verschillende features met de Survival feature bekijken. Zonder aanpassingen aan de data kunnen we dit alleen doen voor features zonder null-waarden. Daarnaast heeft het alleen zin om dit te doen op categorische (Sex), ordinale (Pclass) en discrete (SibSP, Parch) data types.

In [ ]:
#Pclass:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Duidelijk is te zien dat de reis klasse een sterke correlatie heeft voor in ieder geval klasse 1 en ook nog voor klasse 2. Andersom is er uiteraard ook een hoge correlatie voor het niet overleven tov de reis klasse (3).

Dit is een feature welke duidelijk gebruikt kan worden voor de voorspelling.

Dan voor "Sex":

In [ ]:
# TODO zelfde voor geslacht

Kijkende naar onze aannames hierboven kunnen we idd zeggen dat vrouwen een (veel) hogere overlevingskans hadden dan mannen. Dit is een feature welke we goed kunnen gebruiken.

Dan SibSp en Parch:

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Uit SibSp en Parch is voor sommige waarden een hogere correlatie te zien maar voor andere waarden is er geen enkele correlatie te zien. Mogelijk kunnen we hier beter, zoals hierboven beschreven, kijken of we hier nieuwe features uit kunnen samen stellen waar meer waarde uit valt te halen

<h4>Visualisatie</h4>
Voor de correlatie tussen numerieke waardes en Survival rate maken we gebruik van visualisatie. Voor Age plotten we een histogram:

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

Wat we hier zien:
<ul>
    <li>Kinderen jonger dan 4 jaar oud hebben een hoge overlevingskans</li>
    <li>Iedereen rond de 80 jaar oud heeft het overleefd</li>
    <li>Tussen de 15 en 25 jaar oud is er een groot aantal van niet-overlevenden</li>
    <li>De grootste groep passagiers is tusssen de 15 en 25 jaar oud</li>
</ul>
Kijkende naar de aannames welke we hierboven hebben gedaan:
<ul>
    <li>We kunnen leeftijd als feature gaan gebruiken</li>
    <li>We moeten proberen de ontbrekende leeftijd waardes (null-values) aan te vullen</li>
    <li>Het heeft zin om de leeftijd feature te groeperen</li>
</ul>

<h5>Combinatie van numerieke en ordinale features</h5>
We gaan nu verschillende features combineren in één plot om onderlinge correlaties te proberen te identificeren:

In [ ]:
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

<ul>
    <li>Klasse 3 heeft ruim de meeste passagiers maar ook ruim het minst aantal overlevenden.</li>
    <li>Alle kinderen in klasse 2 hebben het overleeft. Een redelijk groot deel van de kinderen in klasse 3 heeft het ook overleefd. In klasse 1 kwamen bijna geen kinderen voor</li>
    <li>In klasse 1 zijn relatief de meeste overlevenden zoals ook 1 van onze aannames is</li>
    <li>We zien hier de invloed van leeftijd en klasse gecombineerd op de survival rate</li>
</ul>
Het lijkt er dus op dat we klasse kunnen gebruiken als feature voor ons classificatie probleem

<h5>Correlatie van categorische features</h5>

In [ ]:
ordered_embarked = train_df["Embarked"].value_counts().index
grid = sns.FacetGrid(train_df, row='Embarked', height=2.2, aspect=1.6, row_order=ordered_embarked)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

<ul>
    <li>Behalve voor de passagiers die in Cherbourg (C) zijn ingestapt is er voor vrouwen een hogere survival rate dan voor mannen</li>
    <li>Dat de survival rate voor mannen hoger is dan voor vrouwen voor Embarked = C kan duiden op een correlatie tussen klasse en embarked en tussen klasse en survived (wat we eerder ook al hadden gezien). Het toont niet direct een correlatie aan tussen embarked en survived.</li>
    <li>Mannelijke overlevingskansen voor klasse 3 in Embarked = Q is hoger dan in klasse 2 (voor Embarked = Q)</li>
</ul>
Sex kan als feature gebruikt worden en Embarked ook. Embarked moet echter wel worden aangevuld gezien de ontbrekende waardes

<h5>Correlatie van categorische en numerieke waardes</h5>
We gaan nu de correlatie van Embarked (categorisch, non-numeriek), Fare (Numeriek, continue) en survival (categorisch numeriek):

In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', height=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

<ul>
    <li>Het is te zien dat een hogere Fare, behalve bij Embarked = Q, een hogere overlevingskans oplevert</li>
    <li>Ook hier is weer te zien dat de plaats van embarkment invloed heeft op de survival rate</li>
</ul>
Fare kunnen we meenemen als feature

<h4>Data correctie, features verwijderen, nieuwe features aanmaken</h4>
Als eerste gaan we de features verwijderen waarvan we hebben gezien dat ze geen of weinig bijdrage leveren aan de voorspelling voor overleving. Gezien de eerdere bevindingen gaan we de Cabin en Ticket features verwijderen, uiteraard op de train én test set:

In [ ]:
# TODO verwijder Cabin en Ticket features
combine = [train_df, test_df]

<h5>Aanmaken nieuwe features</h5>
Als eerste kijken we naar het extraheren van de titel uit het naam veld. Als onderdeel van deze oefening kijken we daarna uiteraard ook of er een correlatie tussen de nieuwe feature "title" en survival is te ontdekken.

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

Een aantal van deze titels gaan we nu samenvoegen:

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

Omzetten van de categorische titel waarden in ordinale waarden:

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

Nu het verwijderen van de laatste features welke we niet gaan gebruiken: Name en PassengerId

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

We kunnen zien dat sommige titels een goede voorspelling opleveren voor de survival rate (Miss/Mrs vs Mr).

<h5>Omzetten van categorische waarden in numerieke waarden</h5>
Voor Sex gaan we male/female omzetten in 0/1.

In [ ]:
for dataset in combine:
    # TODO Mappen female -> 1, male -> 0

train_df.head()

<h4>Aanvullen van ontbrekende waarden</h4>
Gezien niet alle features waardes hebben voor elke passagier, gaan we nu een aantal invullingen doen, beginnende met de age feature. Er zijn verschillende methoden voor om ontbrekende waarden aan te vullen, hier kiezen we voor het gebruiken van de correlatie welke we eerder hebben gezien tussen Age, Sex en Pclass

In [ ]:
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

Ter ondersteuning:

In [ ]:
guess_ages = np.zeros((2,3))
guess_ages

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

We gaan nu de Age feature omzetten in een nieuwe AgeBand feature waarbij de Age wordt ingedeeld in 5 leeftijdsgroepen:

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

En gelijk Age omzetten in ordinale waarden:

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

AgeBand was een tijdelijke feature welke we nu niet meer nodig hebben:

In [ ]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

<h5>Toevoegen van een nieuwe feature FamilySize</h5>
Het aantal familieleden wordt bepaald door de som van Parch en SibSp:

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Om het expliciet te maken voegen we ook een feature "IsAlone" toe om aan te geven of een persoon alleen op de boot zit of niet:

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

De nieuwe feature IsAlone gaan we gebruiken in plaats van FamilySize, Parch en SibSp. Deze laatste 2 gaan we dan ook uit de set verwijderen:

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

De laatste feature welke we gaan toevoegen is Age*Class

In [ ]:
for dataset in combine:
    #TODO feature "Age*Class" toevoegen

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

<h5>Aanvullen van Embarked</h5>
Ook Embarked heeft ontbrekende waarden. Gezien het om 2 records gaat, kiezen we hier voor een andere strategie om deze op te vullen, namelijk gewoon met de meest voorkomende waarden:

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    # TODO Alle nan's vullen met bovenstaande
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Nu alleen de categorische waarden in Embarked omzetten naar numeriek:

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

<h5>Aanvullen van Fare</h5>
Ook hier vullen we de ontbrekende waarde simpel aan, deze keer met de mediaan:

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

Ook voor Fare gaan we een bandbreedte definieren:

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

En dus ook weer omzetten naar numerieke waarden

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

En ook de test set:

In [ ]:
test_df.head(10)

<h2>Voorspellen!</h2>
Het heeft even geduurd maar eindelijk zijn de datasets klaar om te gebruiken voor voorspelling.

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

We gebruiken nog eerst een regressie voor het valideren van onze aannames voor het aanmaken van nieuwe features en om een overzicht te krijgen welke features sterk bijdragen aan het wel of niet overleven

In [ ]:
logreg = LogisticRegression(solver="lbfgs")
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

Hierboven is te zien dat als de waarde van Sex verhoogt (0/man naar 1/vrouw), de survival rate het sterkste toeneemt (kans op overleven). Andersom, als de waarde voor Pclass hoger wordt, dan wordt neemt de overlevingskans het sterkste af. De zelf toegevoegde feature Age*Class is ook een goede (negatieve) voorspeller voor de overlevingskans

<h5>Support Vector Machines</h5>
Nu de voorspelling met lineaire SVM

In [ ]:
svc = SVC(kernel="rbf", gamma="auto")
svc.fit(X_train, Y_train)

# TODO Voorspel de labels voor de test set

acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
print (acc_svc)

<h4>Next steps</h4>
<ul>
    <li>Verbeteren van bovenstaande oplossing</li>
    <li>Denk dan aan het toepassen van cross validation</li>
    <li>Denk dan aan parameter tuning</li>
    <li>Andere kernels?</li>
    <li>Zou jij de ontbrekende data op een andere manier aanvullen?</li>
</ul>